# Imports

In [29]:
import os
import re
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
caminho_base = '/content/drive/MyDrive/red_pill_analises/classificador/oficial_videos_classificacao/folds'
caminho_saida = '/content/drive/MyDrive/red_pill_analises/classificador/oficial_videos_classificacao/folds/folds_teste_llama_limpos'

# Declaração de Funções

In [31]:
def limpeza(texto):
    if not isinstance(texto, str):
        return texto
    #só separando os domínios curtinhos para fazer um tratamento estratégico
    dominios_curtos = [r"hotm.art", r"amzn.to", r"bit.ly", r"t.me", r"youtu.be", r"abre.ai", r"kwai-video.com"]
    outros_termos = [t for t in termos_para_remover if t not in dominios_curtos]

    padrao_geral = re.compile(r'(' + '|'.join(map(re.escape, outros_termos)) + r')', flags=re.IGNORECASE)
    texto = padrao_geral.sub('', texto)

    #retira tudo que está junto com o termo até o espaço
    for dom in dominios_curtos:
        texto = re.sub(re.escape(dom) + r'\S*', '', texto, flags=re.IGNORECASE)

    #remove espaços inúteis
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# Termos a serem removidos

In [32]:
termos_para_remover = [
    #raw string só para evitar erros com strings que possam ser interpretadas como código ou comando para o python

    r"ative o sininho de notificações", r"para não perder nenhum vídeo",
    r"seja bem vindo a mais um", r"inscrever no canal e deixar o like",
    r"Pare de ser feito de Otário!", r"Adquira o Antiotário",
    r"O livro que te ensina a se impor, evitar ciladas e nunca mais ser feito de trouxa",
    r"Evitar manipulações e jogos emocionais", r"Se posicionar com inteligência nos relacionamentos",
    r"Torne-se a melhor versão de si mesmo", r"Privadoesse é apenas",
    r"📕 Aprenda como conquistar qualquer mulher"

    r"O Caminho do Homem Superior", r"The Unplugged Alpha", r"Sexual Utopia in Power",
    r"Why Men Love Bitches", r"LIVRO HACKEANDO O MERCADO SEXUAL", r"No More Mr. Nice Guy",
    r"The Way of the Superior Man", r"CURSO BITCOIN", r"COMBO JOVEM EMPRESÁRIO", r"Combo Manual",

    r"red pill feminina", r"comportamento red pill", r"autores red pill",
    r"canal feminino red pill", r"olhar de homem red pill", r"relacionamento red pill",

    r"📕 - AntiOtário", r"🔱 - Código Masculino", r"🚩 - Red Flags", r"🔞Antiporn",
    r"🧘 - Lições Perpétuas", r"📚", r"Grupo Privado", r"📚", r"🔞 - Antiporn*",
    r"🗿 - Antiotario", r"🗿 Antiotario", r"🔱 - Masculino", r"Manual AntiOtário",
    r"📕 Manual AntiOtário✅", r"📕- ANTIOTÁRIO", r"📕- ANTIOTÁRIO:", r"📕 - Antiotario🔐",
    r"📕 - Antiotario", r"📕 Descubra o poder do Método Alpha:",

    r"#relacionamento", r"#namoro", r"#casal", r"#casamento", r"#casamentonapraia", r"#amor",
    r"#reconquistar", r"#mulher", r"#homem", r"#homens", r"#masculino", r"#masculinidade",
    r"#homemdevalor", r"#homemdealtovalor", r"#homemmoderno", r"#desenvolvimentomasculino",
    r"#motivacao", r"#morenotatuado", r"#LiberdadeMasculina", r"#RedPill", r"#redpill",
    r"#redpillbrasil", r"#redpilled", r"#mentalidadeblack", r"#mestredablack", r"#hackeandoamatrix",
    r"#VSM", r"#sigma", r"#amelhorredpillvemdelas", r"#machoalpha", r"#machoalfa", r"#macho",
    r"#machomacho", r"#seducao", r"#seduçao", r"#conquistarmulheres", r"#conquistarmulher",
    r"#conquistarumamor", r"#comoconquistarumhomem", r"#namorosafado", r"#traicao", r"#traição",
    r"#traiçao", r"#traicão", r"#rafaelaires", r"#antiotario", r"#fabricioacunha",
    r"#fabricioacunhamentor", r"#Miqueinha", r"#shorts", r"#cortespodcast", r"#the_redpill"

    r"➡️ https://manualantiotario.com/AntiOtário",
    r"https://www.google.com/url?sa=E&source=gmail&q=pay.kiwify.com.br",
    r"https://www.google.com/url?sa=E&source=gmail&q=go.hotmart.com",
    r"https://www.google.com/url?sa=E&source=gmail&q=karinefloriana.com.br",
    r"hotm.art", r"amzn.to", r"bit.ly", r"t.me", r"youtu.be", r"abre.ai", r"kwai-video.com"
]

In [33]:
categorias = ['MISOGINIA', 'TOXICIDADE']
numeros = range(1, 6)
arquivos = [f"videos_fold_{i}_estratificado_{cat}.csv" for cat in categorias for i in numeros]

# Salva novos folds em um csv

Teste 1: apenas termos da lista no google docs

In [34]:
# 2. Verificação de segurança: O código só começa se a pasta existir
if not os.path.exists(caminho_saida):
    print(f"❌ Erro: A pasta '{caminho_saida}' não foi encontrada no Drive.")
    print("Verifique se o nome está correto ou se a sincronização do Drive terminou.")
else:
    print(f"✅ Pasta de saída detectada. Iniciando processamento dos {len(arquivos)} arquivos...")

    for nome_arquivo in arquivos:
        path_in = os.path.join(caminho_base, nome_arquivo)

        # --- AQUI É ONDE ADICIONAMOS "LIMPO" NO NOME ---
        # Substituímos '.csv' por '_LIMPO.csv'
        nome_arquivo_limpo = nome_arquivo.replace('.csv', '_LIMPO.csv')
        path_out = os.path.join(caminho_saida, nome_arquivo_limpo)

        if os.path.exists(path_in):
            print(f"Limpando e renomeando: {nome_arquivo} -> {nome_arquivo_limpo}")
            df = pd.read_csv(path_in)

            cols = df.select_dtypes(include=['object']).columns
            for c in cols:
                df[c] = df[c].apply(limpeza)

            # Salva com o novo nome
            #df.to_csv(path_out, index=False, encoding='utf-8-sig')
        else:
            print(f"⚠️ Arquivo original não encontrado: {nome_arquivo}")

    print("\n🚀 Todos os folds foram processados e salvos com sucesso!")

✅ Pasta de saída detectada. Iniciando processamento dos 10 arquivos...
Limpando e renomeando: videos_fold_1_estratificado_MISOGINIA.csv -> videos_fold_1_estratificado_MISOGINIA_LIMPO.csv
Limpando e renomeando: videos_fold_2_estratificado_MISOGINIA.csv -> videos_fold_2_estratificado_MISOGINIA_LIMPO.csv
Limpando e renomeando: videos_fold_3_estratificado_MISOGINIA.csv -> videos_fold_3_estratificado_MISOGINIA_LIMPO.csv
Limpando e renomeando: videos_fold_4_estratificado_MISOGINIA.csv -> videos_fold_4_estratificado_MISOGINIA_LIMPO.csv
Limpando e renomeando: videos_fold_5_estratificado_MISOGINIA.csv -> videos_fold_5_estratificado_MISOGINIA_LIMPO.csv
Limpando e renomeando: videos_fold_1_estratificado_TOXICIDADE.csv -> videos_fold_1_estratificado_TOXICIDADE_LIMPO.csv
Limpando e renomeando: videos_fold_2_estratificado_TOXICIDADE.csv -> videos_fold_2_estratificado_TOXICIDADE_LIMPO.csv
Limpando e renomeando: videos_fold_3_estratificado_TOXICIDADE.csv -> videos_fold_3_estratificado_TOXICIDADE_LIMPO

Teste 2: Remove todas as #'s e links

In [35]:
# Configuração do diretório de saída para a segunda limpeza (remoção de hashtags/URLs)
caminho_saida_segunda_limpeza = os.path.join(caminho_base, 'folds_teste_llama_limpos_v2')
os.makedirs(caminho_saida_segunda_limpeza, exist_ok=True)

# O diretório de entrada para esta segunda limpeza será o diretório de saída da primeira limpeza
caminho_entrada_primeira_limpeza = '/content/drive/MyDrive/red_pill_analises/classificador/oficial_videos_classificacao/folds/folds_teste_llama_limpos'

def limpar_texto(texto):
    """
    Remove URLs, hashtags completas e normaliza espaços.
    """
    if not isinstance(texto, str):
        return texto

    # 1. Remove URLs (http, https, www)
    # AJUSTE: \S* permite remover URLs mesmo que estejam incompletas (ex: apenas "https://")
    texto = re.sub(r'https?://\S*|www\.\S*', '', texto)

    # 2. Remove hashtags inteiras (ex: "#mulher" vira vazio)
    #texto = re.sub(r'#\S+', '', texto)

    # 3. Remove termos específicos
    if 'termos_para_remover' in globals():
        padrao = r'|'.join(map(re.escape, termos_para_remover))
        texto = re.sub(padrao, '', texto, flags=re.IGNORECASE)

    # 4. Remove espaços duplos e nas extremidades
    return re.sub(r'\s+', ' ', texto).strip()

def processar_arquivos(lista_arquivos, dir_entrada, dir_saida):
    print(f"🚀 Iniciando processamento de {len(lista_arquivos)} arquivos...")

    for arquivo_base_nome in lista_arquivos:
        # O nome real do arquivo de entrada já terá o sufixo _LIMPO.csv
        path_in = os.path.join(dir_entrada, arquivo_base_nome)

        if not os.path.exists(path_in):
            print(f"⚠️ Arquivo ignorado (não encontrado): {arquivo_base_nome}")
            continue

        # Carregamento e Limpeza
        df = pd.read_csv(path_in)

        # Aplica limpeza apenas em colunas de texto
        cols_texto = df.select_dtypes(include=['object']).columns
        for col in cols_texto:
            df[col] = df[col].apply(limpar_texto)

        # Salvamento
        nome_arquivo_saida = arquivo_base_nome.replace('.csv', '_v2.csv')
        path_out = os.path.join(dir_saida, nome_arquivo_saida)

        df.to_csv(path_out, index=False, encoding='utf-8-sig')
        print(f"✅ Salvo: {nome_arquivo_saida}")

# Gera a lista de arquivos de entrada para esta segunda limpeza.
arquivos_para_segunda_limpeza = [f.replace('.csv', '_LIMPO.csv') for f in arquivos]

# Execução da segunda limpeza
processar_arquivos(arquivos_para_segunda_limpeza, caminho_entrada_primeira_limpeza, caminho_saida_segunda_limpeza)

🚀 Iniciando processamento de 10 arquivos...
✅ Salvo: videos_fold_1_estratificado_MISOGINIA_LIMPO_v2.csv
✅ Salvo: videos_fold_2_estratificado_MISOGINIA_LIMPO_v2.csv
✅ Salvo: videos_fold_3_estratificado_MISOGINIA_LIMPO_v2.csv
✅ Salvo: videos_fold_4_estratificado_MISOGINIA_LIMPO_v2.csv
✅ Salvo: videos_fold_5_estratificado_MISOGINIA_LIMPO_v2.csv
✅ Salvo: videos_fold_1_estratificado_TOXICIDADE_LIMPO_v2.csv
✅ Salvo: videos_fold_2_estratificado_TOXICIDADE_LIMPO_v2.csv
✅ Salvo: videos_fold_3_estratificado_TOXICIDADE_LIMPO_v2.csv
✅ Salvo: videos_fold_4_estratificado_TOXICIDADE_LIMPO_v2.csv
✅ Salvo: videos_fold_5_estratificado_TOXICIDADE_LIMPO_v2.csv


# Teste de Sanidade

In [36]:
import pandas as pd
import os
import re

def realizar_teste_sanidade(caminho_pasta, lista_termos):
    print(f"🔍 Verificando arquivos em: {caminho_pasta}")
    print(f"📊 Total de termos para conferir: {len(lista_termos)}\n")
    print("-" * 60)

    # Filtra apenas os arquivos que têm o sufixo _LIMPO
    arquivos_limpos = [f for f in os.listdir(caminho_pasta) if f.endswith('_LIMPO.csv')]

    if not arquivos_limpos:
        print("❌ Nenhum arquivo com o sufixo '_LIMPO.csv' foi encontrado!")
        return

    for arq in arquivos_limpos:
        caminho_completo = os.path.join(caminho_pasta, arq)
        df = pd.read_csv(caminho_completo)

        # Seleciona apenas colunas de texto
        colunas_texto = df.select_dtypes(include=['object']).columns
        total_conflitos = 0
        detalhes_conflitos = []

        for col in colunas_texto:
            for termo in lista_termos:
                # Busca exata (ignore_case) para ver se o termo ainda existe
                # Usamos re.escape para lidar com os emojis e símbolos da sua lista
                encontrados = df[col].str.contains(re.escape(termo), case=False, na=False).sum()

                if encontrados > 0:
                    total_conflitos += encontrados
                    detalhes_conflitos.append(f"   - Termo '{termo}': {encontrados} ocorrência(s)")

        # Resultado por arquivo
        if total_conflitos == 0:
            print(f"✅ {arq:<50} | STATUS: 100% LIMPO")
        else:
            print(f"⚠️ {arq:<50} | STATUS: {total_conflitos} SOBRAS")
            for detalhe in detalhes_conflitos:
                print(detalhe)

    print("-" * 60)
    print("\nTeste concluído!")

# Chamada da função (certifique-se de que termos_para_remover está definida no seu notebook)
realizar_teste_sanidade(caminho_saida, termos_para_remover)

🔍 Verificando arquivos em: /content/drive/MyDrive/red_pill_analises/classificador/oficial_videos_classificacao/folds/folds_teste_llama_limpos
📊 Total de termos para conferir: 108

------------------------------------------------------------
⚠️ videos_fold_1_estratificado_MISOGINIA_LIMPO.csv    | STATUS: 122 SOBRAS
   - Termo '📕- ANTIOTÁRIO': 5 ocorrência(s)
   - Termo '📕- ANTIOTÁRIO:': 5 ocorrência(s)
   - Termo '📕 - Antiotario🔐': 12 ocorrência(s)
   - Termo '📕 - Antiotario': 12 ocorrência(s)
   - Termo '📕 Descubra o poder do Método Alpha:': 6 ocorrência(s)
   - Termo '#relacionamento': 82 ocorrência(s)
⚠️ videos_fold_2_estratificado_MISOGINIA_LIMPO.csv    | STATUS: 126 SOBRAS
   - Termo '📕- ANTIOTÁRIO': 7 ocorrência(s)
   - Termo '📕- ANTIOTÁRIO:': 7 ocorrência(s)
   - Termo '📕 - Antiotario🔐': 6 ocorrência(s)
   - Termo '📕 - Antiotario': 6 ocorrência(s)
   - Termo '📕 Descubra o poder do Método Alpha:': 6 ocorrência(s)
   - Termo '#relacionamento': 94 ocorrência(s)
⚠️ videos_fold_3_estr